# Imports



In [ ]:
import os
import requests
import json
import pandas as pd
import re
from urllib. parse import urlparse                                              # Para extraer el dominio de una url
import matplotlib.pyplot as plt
from pandas import json_normalize

import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from string import punctuation                                                  # Signos de puntuación
from nltk.corpus import stopwords                                               # StopWords
from nltk.sentiment.vader import SentimentIntensityAnalyzer                     # Análisis de sentimiento
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from sklearn.preprocessing import MinMaxScaler
from nltk.stem.porter import *                                                  # Stemming

from sklearn import preprocessing

# Dimensionality reduction libraries
from sklearn.decomposition import PCA

import gensim

# Librerías de cada uno de los modelos de aprendizaje no supervisado que se van a utilizar durante el proyecto
from sklearn.cluster import KMeans, AffinityPropagation, AgglomerativeClustering, Birch, DBSCAN, MiniBatchKMeans, MeanShift, OPTICS, SpectralClustering
from sklearn.mixture import GaussianMixture

from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabasz_score, davies_bouldin_score

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Recolección de datos

Definición de token de acceso a la API de Twitter

In [ ]:
os.environ['BEARER_TOKEN'] ='AAAAAAAAAAAAAAAAAAAAADTYSgEAAAAAeZ3oZmAAU5Wy%2BD8JDb4oufgY4cU%3DsJ09bufT525BDLTwI6DsDW4rCuo57Y5eXUJFwhIRkouvMZbQGH'

def auth():
    return os.getenv('BEARER_TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

Método para crear la url de las diferentes peticiones que se van a realizar a la API de Twitter para la búsqueda de noticias, en ella se seleccionan los diferentes campos a rellenar en la query a realizar a dicha API, y que se mencionan a continuación:

- La variable 'keyword' relativa a la query representa el filtro utilizado en la búsqueda de noticias, el cual es incluido como atributo de creación de la url de la petición.

- Las variables 'start_date' y 'end_date' también se introducen como atributos de la función de creación de la url, y representan el momento a partir del cual empezar a recopilar tweets, y el momento en el que dejar de recopilarlos.

- La variable 'max_results' también se introduce como atributo de la función y sirve para indicar un número concreto de tweets a recopilar, el cual esta inicializado a 10, y tiene como máximo valor, debido a las limitaciones que presenta la API de Twitter en su formato de developer para estudiantes, de 500 peticiones por cada query.

El resto de parámetros que contiene la query representan los datos que se solicita que devuelva dicha query, en este caso, el id del autor del tweet (author_id), el texto del tweet (text), el momento en el que se publicó dicho tweet (created_at)... Estos parámetros pueden ser modificados, incluyendo o sustituyendolos por los aportados como comentario para ampliar o modificar la información devuelta por la query.


In [ ]:
def create_url(keyword, start_date,end_date,max_results = 10):

    search_url = "https://api.twitter.com/2/tweets/search/all"      # Change to the endpoint you want to collect data from

    # Change params based on the endpoint you are using
    query_params = {'query': keyword, 
                    'start_time': start_date, 
                    'end_time': end_date, 
                    'max_results': max_results, 
                    'expansions': 'author_id', #in_reply_to_user_id, author_id, geo.place_id',
                    'tweet.fields': 'text', # id,text, author_id, in_reply_to_user_id, geo, conversation_id, created_at,lang,public_metrics, referenced_tweets, reply_settings, source',
                    'tweet.fields': 'created_at',
                    #'tweet.fields': 'public_metrics',
                    'user.fields': 'name',#id, name, username, created_at, description, public_metrics, verified',
                    'user.fields': 'verified',
                    'user.fields': 'public_metrics',
                    'place.fields': 'country',# full_name, id, country, country_code, geo, name, place_type',
                    'next_token': {}}
    
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

Definición de las variables que se van a introducir como atributos a la función de creación de la query a la API de Twitter

In [ ]:
#Recolección de noticias en inglés
indice = 0
dfen = pd.DataFrame()

while indice < 50000:                                                           # Bucle para la recopilación de un total de 50.000 tweets (El máximo que se permite con los permisos actuales en la API de Twitter antes de un periodo de espera es de 150.000)

  # Definición de las variables para crear la URL de la petición a la API de Twitter
  bearer_token = auth()
  headers = create_headers(bearer_token)
  keyword = "#news -is:retweet has:links lang:en"                               # Filtros: Tweets con hashtag = "news" (#news), que NO sean retweets (-is:retweet), que contengan links (has:links) y estén en inglés (lang:en)
  start_time = "2022-01-01T00:00:00.000Z"                                       # Fecha de inicio de recolección de tweets
  end_time = "2022-05-12T00:00:00.000Z"                                         # Fecha del último tweet posible
  max_results = 500                                                             # Máxima cantidad de tweets que puede devolver
  
  if indice != 0:
    end_time = df1['created_at'][len(df1)-1]
  
  # Creación de la petición a la API de Twitter
  url = create_url(keyword, start_time, end_time, max_results)                  # Creación de la URL
  json_response = connect_to_endpoint(url[0], headers, url[1])
  i = json.dumps(json_response, indent=4, sort_keys=True)                       # Recolección de la respuesta de la API de Twitter en un json


  # Transformación del JSON devuelto por la API de Twitter en un DataFrame de la librería Pandas
  info = json.loads(i)
  df1 = pd.DataFrame()                                                          # Al encontrarse los diferentes campos de los tweets que se desean obtener, en diferentes niveles del fichero JSON, se ha decidido extraerlos en 3 DataFrames diferentes para cada nivel, para posteriormente unirlos en un único DataFrame final
  df1 = json_normalize(info['data'])
  df2 =json_normalize(info['includes'].values())
  df3 = pd.DataFrame()
  for j in range (0, df2.size):
    d = pd.DataFrame.from_dict(df2[j][0], orient='index').T
    df3 = df3.append(d)

  # Normalización de DataFrame
  df3 = df3.reset_index()
  df3 = df3.drop(['index'], axis=1)

  # Dataframes unidos
  df1['name'] = ''
  df1['username'] = ''
  df1['followers_count'] = ''
  df1['following_count'] = ''
  df1['listed_count'] = ''
  df1['tweet_count'] = ''
  index1 = 0
  for j in df1['author_id']:

    index2 = 0
    for i in df3['id']:
      if j == i:
        df1['name'][index1] = df3['name'][index2]
        df1['username'][index1] = df3['username'][index2]
        df1['followers_count'][index1] = df3['public_metrics.followers_count'][index2]
        df1['following_count'][index1] = df3['public_metrics.following_count'][index2]
        df1['listed_count'][index1] = df3['public_metrics.listed_count'][index2]
        df1['tweet_count'][index1] = df3['public_metrics.tweet_count'][index2]
        
      index2 += 1 
    index1 += 1

  last_tweet = df1['created_at'][df1.shape[0]-1]
  indice = indice + df1.shape[0]
  dfen = dfen.append(df1, ignore_index=True)

Endpoint Response Code: 200


# Extracción de nuevas variables


Método para extraer las urls contenidas en un texto

In [ ]:
def url_redirect(string):
    """Extracts redirection url"""
    url = re.findall("(?P<url>https?://[^\s]+)", string) 
    return url

Método que elimina las urls contenidas en un texto

In [ ]:
def url_erase(string):
    """Eliminates urls from text"""
    r = re.sub(r"http\S+", "", string)
    return r

A continuación, se extraen las urls contenidas en cada uno de los tweets y se almacenan en una nueva columna llamada 'twitter_url', para posteriormente eliminar las urls del texto de los tweets para que no influyan en el análisis textual


In [ ]:
dfen['twitter_url'] = dfen['text'].apply(url_redirect)
dfen['clean_text'] = dfen['text'].apply(url_erase)

Twitter presenta la peculiaridad de que todas las urls que se publican en sus tweets, son presentadas por medio de un acortador propio de Twitter por motivos de seguridad. Como el objetivo es extraer estas urls de cada una de las noticias recopiladas para extraer posteriormente el dominio de cada una de ellas, el cual representa el periodico digital fuente de cada noticia, es necesario crear un código que extraiga las urls originales de los acortadores propios de Twitter

Debido al gran volúmen de datos que se tiene y la complejidad computacional que requiere ejecutar el siguiente código, Google Colab se bloquea, por lo que se decidió realizar guardados periodicos del dataset parcial obtenido e ir ejecutando este código en pequeños sets.

En el siguiente código se muestra el código original sin guardados, como si fuera aplicado a todo el dataset original, deberá ser adaptado si se tiene el mismo problema computacional


In [ ]:
indice = 0
dfen['urls'] = ''
for i in dfen['twitter_url']:
  if indice < 50000:
     try:
      r = requests.get(i[0]) 
      dfen['urls'][indice] = r.url
      indice = indice +1
     except:
      dfen['urls'][indice] = '' 
      indice = indice +1
  else: 
      break

df = dfen

Se eliminan los tweets de los cuales no se ha conseguido obtener una url

In [ ]:
import numpy as np
df['urls'].replace('', np.nan, inplace=True)
df = df.dropna()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Método de extracción del dominio de una URL para utilizarlo como una nueva variable en el DataFrame

In [ ]:
def clean_urls(string):
    domain = urlparse(string).netloc
    return domain

In [ ]:
df['clean_urls'] = df['urls'].apply(clean_urls)

Método para la aplicación del modelo de análisis de sentimientos sobre texto pasado como parámetro

In [ ]:
sid = SentimentIntensityAnalyzer()
def vader_sentiment(text):
    """ Calculate and return the nltk vader (lexicon method) sentiment """
    return sid.polarity_scores(text)['compound']

Aplicación del método de análisis de sentimiento sobre la variable textual de cada uno de los tweets del DataFrame

In [ ]:
df['vader compound'] = df['clean_text'].apply(vader_sentiment)

In [ ]:
def count_words(text):
    """Devuelve el número de palabras que contiene cada tweet"""
    return len(text.split())

Se aplica el contador de palabras de cada tweet y se almacena el resultado en una nueva columna

In [ ]:
df['num_words'] = df['text'].apply(count_words)

Se crea un escalador para convertir el número de palabras de cada tweet en un valor comprendido entre [-1,1]

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
columns_to_scale = ['num_words']
scaler.fit(df[columns_to_scale])
df[columns_to_scale] = scaler.transform(df[columns_to_scale])

# Preprocesamiento textual y Word Embedding

Se aplica el tokenizado, y la eliminación de stopwords y signos de puntuación sobre el texto

In [ ]:
punctuation = list(punctuation)
stop_words = set(stopwords.words('english'))
vocab = []
df['tokens'] = ''
for tweet in range(0,49097):    
  word_tokens = word_tokenize(df['clean_text'][tweet])
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    
  filtered_sentence = []
    
  for w in word_tokens:
      if (w not in stop_words and w not in punctuation):
          filtered_sentence.append(w)
          if w not in vocab:
            vocab.append(w)
  df['tokens'][tweet] = filtered_sentence

['Cramer', "'s", 'lightning', 'round', ':', 'I', 'like', 'Blackstone', 'over', 'KKR', '#', 'news', '#', 'topstories', '#', 'berkleybearnews']
['Cramer', "'s", 'lightning', 'round', 'I', 'like', 'Blackstone', 'KKR', 'news', 'topstories', 'berkleybearnews']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Se crea un Stemmer de tipo Porter, para realizar el proceso de Stemming sobre cada uno de los tokens creados anteriormente a partir del texto

In [ ]:
stemmer = PorterStemmer() 
df['tokenize_stemm'] = df['tokens'].apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

# Volvemos a juntar los tokens
for i in range(len(df['tokenize_stemm'])):
    df['tokenize_stemm'][i] = ' '.join(df['tokenize_stemm'][i])    
df['tidy_tweet'] = df['tokenize_stemm']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Algunas urls de las obtenidas anteriormente, son enlaces a grupos o cuentas de telegram, identificadas por un acortador 't.me', por lo que estos tweets que presentan esta característica son descartados del dataset ya que no aportan información sobre el origen de la noticia


In [ ]:
for i in range(0,25):
  if 't.me' in df['clean_urls'][i]:
    df = df.drop(i, axis=0)
df.reset_index(inplace=True,drop=True)
df = df.drop(['index'],axis=1)


KeyError: ignored

Se tokeniza de nuevo el texto de cada tweet tras haber aplicado el stemming

In [ ]:
df['tokenized_tweet'] = df['tidy_tweet'].apply(lambda x: x.split()) # tokenizing 

Se selecciona un pequeño conjunto del dataset original para ser utilizado como dataset de validación, el cual va a ser etiquetado manualmente, para posteriormente comprobar si los resultados del clustering han sido acertados

In [ ]:
valid = df.iloc[:10,:] #99,:]

Se crea el modelo de Word2Vec que se va a utilizar para convertir cada uno de los tweets en un vector que represente el texto de esos tweets

In [ ]:
model_w2v = gensim.models.Word2Vec(
            df['tokenized_tweet'],
            size=500, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 

model_w2v.train(df['tokenized_tweet'], total_examples= len(df['tidy_tweet']), epochs=20)

(429, 6520)

Una vez definido y entrenado el modelo Word2Vec con nuestro conjunto de datos textuales, se crea este método para generar el vector de representación de cada tweet, y posteriormente, se aplica y se guarda en formato DataFrame para poder ser añadido al resto de datos que se tienen del dataset original

In [ ]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
wordvec_arrays = np.zeros((len(df['tokenized_tweet']), 500)) 
for i in range(len(df['tokenized_tweet'])):
    wordvec_arrays[i,:] = word_vector(df['tokenized_tweet'][i], 500)
wordvec_df = pd.DataFrame(wordvec_arrays)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


# Preprocesamiento de datos categóricos

Se seleccionan todas las columnas del dataset original que van a ser utilizadas para el análisis, menos la parte textual que se encuentra ya almacenada en un DataFrame aparte, como se ha visto en la celda anterior

In [ ]:
features = ['num_words','username','vader compound','clean_urls','author_id','followers_count', 'following_count', 'listed_count', 'tweet_count']
dataset = df[features]

Se aplica un Label Encoder para transformar cada una de las columnas que contienen datos de tipo categórico textual en columnas con datos numéricos que representan dichos mismos datos categóricos

In [ ]:
le = preprocessing.LabelEncoder()
dataset["username_labeled"] = le.fit_transform(dataset["username"])
dataset['clean_urls_labeled'] = le.fit_transform(dataset['clean_urls'])
dataset.drop(['username','clean_urls'], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,num_words,vader compound,author_id,followers_count,following_count,listed_count,tweet_count,username_labeled,clean_urls_labeled
0,0.310345,0.0000,1168765575654453248,20,12,0,11291,17,15
1,0.034483,0.0000,1379367732383154177,158,535,2,95451,10,5
2,-0.655172,0.5106,454478622,541,525,223,124912,11,14
3,0.448276,-0.6369,1252396121294602240,34,4,0,55197,18,18
4,-0.586207,0.0000,1514149588252385286,1825,4303,0,46152,2,2
5,0.793103,-0.3400,3265583034,3547,1815,143,126251,0,0
6,-0.448276,0.0000,1125833315402100740,557,945,0,4630,3,16
7,-0.586207,0.4926,3849691696,690,1850,3,7036,6,7
8,-0.103448,0.0000,1379367732383154177,158,535,2,95451,10,5
9,-0.103448,0.0000,1031294099696050176,378,84,10,19359,4,1


Se realiza un escalado de todas las columnas de datos categórigcos a un rango de valores comprendido entre [-1,1] al igual que se realizó previamente con la columna que contiene el número de palabras de cada tweet, y como es representado también por la columna de análisis de sentimiento generada con anterioridad

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
columns_to_scale = ['username_labeled', 'clean_urls_labeled','followers_count', 'following_count', 'listed_count', 'tweet_count']
scaler.fit(dataset[columns_to_scale])
dataset[columns_to_scale] = scaler.transform(dataset[columns_to_scale])
dataset = dataset.drop(['username','clean_urls'], axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Unión de los dos dataset generados, el que contiene los datos categórigcos ya preprocesados y escalados, y el que contiene la representación vectorial de cada uno de los tweets

In [ ]:
dat = pd.concat([dataset, wordvec_df], axis=1)

Se aplica un Principal Component Analysis para eliminar las variables del dataset que aportan menor información al análisis

In [ ]:
pca = PCA(n_components = 255)
p = pca.fit_transform(dat)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


ValueError: ignored

# Modelos de aprendizaje y resultados

A continuación se van a ejecutar los diferentes modelos que se van a probar en el análisis además de mostrar los resultados que presentan cada uno de ellos


**Kmeans**

Se ejecuta el modelo Kmeans buscando en este caso 3 clusters diferentes, que representan las noticias falsas, las verdaderas y un tercer grupo que son noticias de propaganda que no son en su completitud falsas

In [ ]:
kmeans = KMeans(3, init='random')
label = kmeans.fit_predict(p)

Impresión de gráfico con los diferentes clusters detectados

In [ ]:
u_labels = np.unique(label)
 
for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**Affinity Propagation**

Se ejecuta el modelo de Propagación de afinidad al cual no se le introducen el número de clusters que se quiere identificar, de manera que lo que se busca mediante este modelo es ver cuantos clusters diferentes es capaz de identificar

In [ ]:
ap = AffinityPropagation(damping=0.9,random_state=5).fit(p)
label = ap.predict(p)

In [ ]:
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**Aglomerative Clustering**

Se ejecuta el modelo de Clustering aglomerativo al cual sí hay que proporcionarle el número de clusters que se desea obtener

In [ ]:
agC = AgglomerativeClustering(3)
label = agC.fit_predict(p)

In [ ]:
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**Birch**

Se ejecuta el modelo de Birch al cual sí hay que proporcionarle el número de clusters que se desea obtener

In [ ]:
birch = Birch(n_clusters=3)
label = birch.fit_predict(p)

In [ ]:
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**DBSCAN**

Se ejecuta el modelo de DBSCAN, basado en densidad, al cual no hay que proporcionarle el número de clusters que se desea obtener

In [ ]:
dbscan = DBSCAN()
label = birch.fit_predict(p)

In [ ]:
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**MeanShift**

Se ejecuta el modelo de MeanShift al cual no hay que proporcionarle el número de clusters que se desea obtener

In [ ]:
mean = MeanShift()
label = mean.fit_predict(p)

In [ ]:
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**Mini Batch Kmeans**

Se ejecuta el modelo de Mini Batch Kmeans al cual sí hay que proporcionarle el número de clusters que se desea obtener

In [ ]:
mini =MiniBatchKMeans(n_clusters=3,random_state=32,batch_size=6)
label = mini.fit_predict(p)

In [ ]:
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**OPTICS**

Se ejecuta el modelo de OPTICS al cual no hay que proporcionarle el número de clusters que se desea obtener

In [ ]:
opt = OPTICS(min_samples=2)
label = opt.fit_predict(p)

/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_optics.py:903: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [ ]:
u_labels = np.unique(label)
 
for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**Spectral Clustering**

Se ejecuta el modelo de Clustering espectral, al cual sí hay que proporcionarle el número de clusters que se desea obtener

In [ ]:
spec = SpectralClustering(n_clusters=3)
label = spec.fit_predict(p)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_spectral_embedding.py:261: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  "Graph is not fully connected, spectral embedding may not work as expected."


In [ ]:
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

**Gaussian Mixture**

Se ejecuta el modelo de Mezcla de gaussianas, al cual no hay que proporcionarle el número de clusters que se desea obtener

In [ ]:
gau = GaussianMixture(n_components=3)
label = gau.fit_predict(p)

In [ ]:
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(p[label == i , 0] , p[label == i , 1] , label = i)
plt.legend()
plt.show()

IndexError: ignored

Cálculo de métricas 

In [ ]:
sil_scores = []
se = []
calinski_score = []
davies_score = []
se.append(kmeans.inertia_)
sil_scores.append(silhouette_score(p, label))
calinski_score.append(calinski_harabasz_score(p, label))
davies_score.append(davies_bouldin_score(p, label))

print('Silhouette score:', sil_scores)

NameError: ignored